In [1]:
import pandas as pd
import duckdb

In [19]:
generales = pd.read_parquet('/Users/agusvaser/Documents/ITBA/2023/2ndo cuatri/Visualización de la Información/Trabajo Practico/ResultadosElectorales_1v.parquet')
paso = pd.read_parquet('/Users/agusvaser/Documents/ITBA/2023/2ndo cuatri/Visualización de la Información/Trabajo Practico/ResultadosElectorales_PASO_2023 (1).parquet')
balotaje = pd.read_parquet('/Users/agusvaser/Documents/ITBA/2023/2ndo cuatri/Visualización de la Información/Trabajo Practico/ResultadosElectorales_2023_SegundaVuelta (1).parquet')

In [3]:
data = duckdb.sql("""select * 
from generales

union 

select *
from paso
""")

In [4]:
generales.head(5)

,año,eleccion_tipo,recuento_tipo,padron_tipo,distrito_id,distrito_nombre,seccionprovincial_id,seccionprovincial_nombre,seccion_id,seccion_nombre,...,mesa_tipo,mesa_electores,cargo_id,cargo_nombre,agrupacion_id,agrupacion_nombre,lista_numero,lista_nombre,votos_tipo,votos_cantidad
0,2023,GENERAL,PROVISORIO,NORMAL,1,Ciudad Autónoma de Buenos Aires,0,None,1,Comuna 01,...,NATIVOS,343,1,PRESIDENTE Y VICE,0,None,0.0,None,NULO,0
1,2023,GENERAL,PROVISORIO,NORMAL,1,Ciudad Autónoma de Buenos Aires,0,None,1,Comuna 01,...,NATIVOS,343,1,PRESIDENTE Y VICE,0,None,0.0,None,IMPUGNADO,0
2,2023,GENERAL,PROVISORIO,NORMAL,1,Ciudad Autónoma de Buenos Aires,0,None,1,Comuna 01,...,NATIVOS,343,1,PRESIDENTE Y VICE,0,None,0.0,None,RECURRIDO,0
3,2023,GENERAL,PROVISORIO,NORMAL,1,Ciudad Autónoma de Buenos Aires,0,None,1,Comuna 01,...,NATIVOS,343,1,PRESIDENTE Y VICE,0,None,0.0,None,COMANDO,0
4,2023,GENERAL,PROVISORIO,NORMAL,1,Ciudad Autónoma de Buenos Aires,0,None,1,Comuna 01,...,NATIVOS,349,1,PRESIDENTE Y VICE,134,UNION POR LA PATRIA,NaN,None,POSITIVO,95


In [5]:
data.columns

['año',
 'eleccion_tipo',
 'recuento_tipo',
 'padron_tipo',
 'distrito_id',
 'distrito_nombre',
 'seccionprovincial_id',
 'seccionprovincial_nombre',
 'seccion_id',
 'seccion_nombre',
 'circuito_id',
 'circuito_nombre',
 'mesa_id',
 'mesa_tipo',
 'mesa_electores',
 'cargo_id',
 'cargo_nombre',
 'agrupacion_id',
 'agrupacion_nombre',
 'lista_numero',
 'lista_nombre',
 'votos_tipo',
 'votos_cantidad']

In [6]:
elecciones_presidenciales = duckdb.sql("""select *
from data
where cargo_id = 1
""")

In [7]:
generales_presidenciales = duckdb.sql("select * from elecciones_presidenciales where eleccion_tipo = 'GENERAL'")
paso_presidenciales = duckdb.sql("select * from elecciones_presidenciales where eleccion_tipo = 'PASO'")

In [8]:
participacion = duckdb.sql("""select eleccion_tipo, agrupacion_nombre,sum(votos_cantidad) as count 
from elecciones_presidenciales
group by 1,2""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [9]:
agrupaciones = duckdb.sql("""select distinct agrupacion_nombre
from elecciones_presidenciales
""").df()

In [10]:
import altair as alt

colorlist=[]

alt.Chart(participacion).mark_bar().encode(
    x="count",
    y='eleccion_tipo',
    # color=alt.Color('agrupacion_nombre').scale(domain=agrupaciones, range=colorlist)
)


alt.Chart(...)

In [11]:
partidos = duckdb.sql("""select distinct agrupacion_id
from generales_presidenciales
""")
partidos

┌───────────────┐
│ agrupacion_id │
│     int64     │
├───────────────┤
│           133 │
│           136 │
│           135 │
│             0 │
│           132 │
│           134 │
└───────────────┘

In [12]:
rendimiento_generales = duckdb.sql("""select agrupacion_nombre as partido, sum(votos_cantidad) as cantidad, (sum(votos_cantidad)/(select sum(votos_cantidad) from generales_presidenciales)*100) as porcentaje
from generales_presidenciales
where agrupacion_nombre is not null
group by agrupacion_nombre
order by 3 desc
""").df()
rendimiento_generales

,partido,cantidad,porcentaje
0,UNION POR LA PATRIA,9645983.0,35.593147
1,LA LIBERTAD AVANZA,7884336.0,29.092766
2,JUNTOS POR EL CAMBIO,6267152.0,23.125446
3,HACEMOS POR NUESTRO PAIS,1784315.0,6.584024
4,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,709932.0,2.619610


In [13]:
rendimiento_paso = duckdb.sql("""select agrupacion_nombre as partido, sum(votos_cantidad) as cantidad, (sum(votos_cantidad)/(select sum(votos_cantidad) from paso_presidenciales)*100) as porcentaje
from paso_presidenciales
where agrupacion_nombre in (select distinct agrupacion_nombre from generales_presidenciales)
group by agrupacion_nombre
order by 3 desc
""").df()
rendimiento_paso

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,partido,cantidad,porcentaje
0,LA LIBERTAD AVANZA,7116352.0,29.630755
1,JUNTOS POR EL CAMBIO,6698029.0,27.888960
2,UNION POR LA PATRIA,6460689.0,26.900734
3,HACEMOS POR NUESTRO PAIS,907437.0,3.778346
4,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,628893.0,2.618557


In [38]:
import altair as alt
import pandas as pd

color_scale = alt.Scale(
    domain=['UNION POR LA PATRIA', 'HACEMOS POR NUESTRO PAIS', 'FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD', 'JUNTOS POR EL CAMBIO', 'LA LIBERTAD AVANZA'],
    range=['#6BADDB', '#70BAA0', '#E88D8D', '#F8D34B', '#B29CD8']
)

# Chart for table1
chart_table1 = alt.Chart(rendimiento_paso).mark_bar().encode(
    alt.X('cantidad:Q'),
    alt.Y('partido:N').sort("-x"),
    color=alt.Color('partido:N', scale=color_scale),
).properties(
    title='Cantidad de votos por partido en las PASO',
    width=300,
    height=100
)


chart_table1

alt.Chart(...)

In [39]:
# Chart for table2
chart_table2 = alt.Chart(rendimiento_generales).mark_bar().encode(
    alt.X('cantidad:Q'),
    alt.Y('partido:N').sort("-x"),
    color=alt.Color('partido:N', scale=color_scale),
).properties(
    title='Cantidad de votos por partido en las Generales',
    width=300,
    height=100
)

chart_table2

alt.Chart(...)

In [23]:
rendimiento_balotaje = duckdb.sql("""select agrupacion_nombre as partido, sum(votos_cantidad) as cantidad, (sum(votos_cantidad)/(select sum(votos_cantidad) from balotaje where votos_tipo = 'POSITIVO')*100) as porcentaje
from balotaje
where votos_tipo = 'POSITIVO'
group by agrupacion_nombre
order by 3 desc
""").df()
rendimiento_balotaje

,partido,cantidad,porcentaje
0,LA LIBERTAD AVANZA,14476462.0,55.694543
1,UNION POR LA PATRIA,11516142.0,44.305457


In [40]:
# Chart for table3

color_scale2 = alt.Scale(
    domain=['LA LIBERTAD AVANZA','UNION POR LA PATRIA'],
    range=['#B29CD8','#6BADDB']
)

chart_table3 = alt.Chart(rendimiento_balotaje).mark_bar().encode(
    alt.X('cantidad:Q'),
    alt.Y('partido:N').sort("-x"),
    color=alt.Color('partido:N', scale=color_scale2),
).properties(
    title='Cantidad de votos por partido en el Balotaje',
    width=300,
    height=100
)

chart_table3

alt.Chart(...)

In [16]:
votos_por_partido_y_provincia = pd.read_csv('/Users/agusvaser/Documents/ITBA/2023/2ndo cuatri/Visualización de la Información/Trabajo Practico/evolucion_elecciones.csv')

In [17]:
votos_bsas = votos_por_partido_y_provincia[votos_por_partido_y_provincia["distrito_nombre"] == "Buenos Aires"]

df_melted = pd.melt(votos_bsas, id_vars=['agrupacion_nombre','distrito_nombre'], value_vars=['porcentaje_generales','porcentaje_paso'],
                    var_name='day', value_name='percentage')

chart = alt.Chart(df_melted).mark_line().encode(
    x=alt.X('day:N', axis=alt.Axis(labelAngle=0, orient='top')),
    y='percentage:Q',
    color=alt.Color('agrupacion_nombre:N', scale=color_scale)
).properties(
    title='Evolucion de Porcentajes del los Partidos en Buenos Aires',
    width=600,
    height=400)

chart


alt.Chart(...)